<a href="https://colab.research.google.com/github/fwangliberty/AIoTDesign-Frontend/blob/master/cnn_small_cicids2018_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection by using small CICIDS 2018 DataSet

In [80]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

## Step 1. Read cleaned CICIDS2018 small dataset

Connect to Google Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [82]:
# load train data
cleanfile = '/content/drive/My Drive/CICIDS2018/xtrain_small.csv'
labelfile = '/content/drive/My Drive/CICIDS2018/ytrain_small.csv'
xtestsmall = '/content/drive/My Drive/CICIDS2018/xtest_small.csv'
ytestsmall = '/content/drive/My Drive/CICIDS2018/ytest_small.csv'
SEED=2

In [81]:
X_train = pd.read_csv(cleanfile, skiprows=0,index_col=0) 
y_train = pd.read_csv(labelfile, skiprows=0,index_col=0)  
X_test = pd.read_csv(xtestsmall, skiprows=0, index_col=0) 
y_test = pd.read_csv(ytestsmall, skiprows=0,index_col=0) 
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,1755722.0,80.0,6.0,9743.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,205.275583,9743.000000,0.000000e+00,9743.0,9743.0,9743.0,9743.00,0.000000e+00,9743.0,9743.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,205.275583,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6156600.0,445.0,6.0,111085.0,6.0,5.0,455.0,338.0,140.0,0.0,75.833333,62.936211,145.0,0.0,67.600000,69.758870,7138.677589,99.023270,11108.500000,1.422834e+04,29284.0,4.0,111051.0,22210.20,1.245618e+04,29356.0,4.0,110986.0,27746.500000,1196.308349,29371.0,26744.0,0.0,0.0,0.0,0.0,132.0,112.0,54.012693,45.010577,0.0,145.0,66.083333,63.402693,4019.901515,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,72.090909,75.833333,67.600000,0.0,0.0,0.0,0.0,0.0,0.0,6.0,455.0,5.0,338.0,8192.0,0.0,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,242193.0,8080.0,6.0,10686.0,3.0,4.0,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,0.0,32.250000,53.767245,42579.075430,655.062699,1781.000000,3.912594e+03,9758.0,21.0,563.0,281.50,3.245620e+02,511.0,52.0,10196.0,3398.666667,5510.518064,9758.0,32.0,0.0,0.0,0.0,0.0,72.0,92.0,280.741157,374.321542,0.0,326.0,56.875000,115.406657,13318.696430,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,65.000000,108.666667,32.250000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,326.0,4.0,129.0,8192.0,219.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6934977.0,22.0,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,285714.285714,7.000000,0.000000e+00,7.0,7.0,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,32.0,142857.142857,142857.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,241.0,230.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4505141.0,80.0,6.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,16.0,5007525.0,1251881.25,2.467885e+06,4953524.0,406.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,8192.0,221.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The small dataset contains an index column. We delete the first column. Finall, the total number of columns is 78.

In [83]:
X_train.drop(X_train.columns[0], axis=1, inplace=True)
X_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,80.0,6.0,9743.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,205.275583,9743.000000,0.000000e+00,9743.0,9743.0,9743.0,9743.00,0.000000e+00,9743.0,9743.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,205.275583,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,445.0,6.0,111085.0,6.0,5.0,455.0,338.0,140.0,0.0,75.833333,62.936211,145.0,0.0,67.600000,69.758870,7138.677589,99.023270,11108.500000,1.422834e+04,29284.0,4.0,111051.0,22210.20,1.245618e+04,29356.0,4.0,110986.0,27746.500000,1196.308349,29371.0,26744.0,0.0,0.0,0.0,0.0,132.0,112.0,54.012693,45.010577,0.0,145.0,66.083333,63.402693,4019.901515,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,72.090909,75.833333,67.600000,0.0,0.0,0.0,0.0,0.0,0.0,6.0,455.0,5.0,338.0,8192.0,0.0,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8080.0,6.0,10686.0,3.0,4.0,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,0.0,32.250000,53.767245,42579.075430,655.062699,1781.000000,3.912594e+03,9758.0,21.0,563.0,281.50,3.245620e+02,511.0,52.0,10196.0,3398.666667,5510.518064,9758.0,32.0,0.0,0.0,0.0,0.0,72.0,92.0,280.741157,374.321542,0.0,326.0,56.875000,115.406657,13318.696430,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,65.000000,108.666667,32.250000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,326.0,4.0,129.0,8192.0,219.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22.0,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,285714.285714,7.000000,0.000000e+00,7.0,7.0,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,32.0,142857.142857,142857.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,241.0,230.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80.0,6.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,16.0,5007525.0,1251881.25,2.467885e+06,4953524.0,406.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,8192.0,221.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
X_test.drop(X_test.columns[0], axis=1, inplace=True)
X_test.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,80.0,6.0,3308.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,6.045949e+02,3.308000e+03,0.000000e+00,3308.0,3308.0,3308.0,3308.0,0.000000e+00,3308.0,3308.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,6.045949e+02,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,443.0,6.0,5049243.0,6.0,4.0,317.0,145.0,235.0,0.0,52.833333,91.693875,145.0,0.0,36.25,72.5,91.498864,1.980495e+00,5.610270e+05,1.654768e+06,4973536.0,13.0,5049243.0,1009848.6,2.243429e+06,5023001.0,26.0,62698.0,2.089933e+04,2.558980e+04,49684.0,726.0,0.0,0.0,0.0,0.0,132.0,84.0,1.188297e+00,0.792198,0.0,235.0,42.000000,77.722584,6040.800000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,46.200000,52.833333,36.25,0.0,0.0,0.0,0.0,0.0,0.0,6.0,317.0,4.0,145.0,8192.0,30016.0,3.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,80.0,6.0,2543935.0,2.0,1.0,8.0,0.0,8.0,0.0,4.000000,5.656854,0.0,0.0,0.00,0.0,3.144734,1.179275e+00,1.271968e+06,1.798831e+06,2543933.0,2.0,2543935.0,2543935.0,0.000000e+00,2543935.0,2543935.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.0,0.0,0.0,0.0,64.0,32.0,7.861836e-01,0.393092,0.0,8.0,4.000000,4.618802,21.333333,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.333333,4.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,1.0,0.0,211.0,219.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52592.0,6.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,2.000000e+06,1.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,0.000000e+00,1.0,1.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,2.000000e+06,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,510.0,-1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80.0,6.0,5005365.0,4.0,4.0,646.0,364.0,646.0,0.0,161.500000,323.000000,364.0,0.0,91.00,182.0,201.783486,1.598285e+00,7.150521e+05,1.864579e+06,4943232.0,16.0,62133.0,20711.0,3.498673e+04,61110.0,402.0,5005341.0,1.668447e+06,2.887131e+06,5002219.0,1015.0,0.0,0.0,0.0,0.0,92.0,92.0,7.991425e-01,0.799143,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,126.250000,161.500000,91.00,0.0,0.0,0.0,0.0,0.0,0.0,4.0,646.0,4.0,364.0,8192.0,221.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# Here we can see the number of rows and columns for each table.
print('Training dataset shape: ', X_train.shape)
print('Testing dataset shape: ', X_test.shape)

Training dataset shape:  (30000, 78)
Testing dataset shape:  (7500, 78)


## Step 2. Normalization

The continuous feature values are normalized into the same feature space. This is important when using features that have different measurements, and is a general requirement of many machine learning algorithms. Therefore, the values for this dataset are also normalized using the Min-Max scaling technique, bringing them all within a range of [0,1].

In [87]:
from sklearn.preprocessing import MinMaxScaler

In [88]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train

array([[1.22085215e-03, 3.52941176e-01, 8.11833367e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.79099011e-03, 3.52941176e-01, 9.25700039e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.23306068e-01, 3.52941176e-01, 8.90416704e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [8.32727994e-01, 3.52941176e-01, 1.25000005e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.22085215e-03, 1.00000000e+00, 9.97585775e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.79099011e-03, 3.52941176e-01, 9.14219205e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [89]:
X_train.shape

(30000, 78)

In [90]:
X_test = scaler.fit_transform(X_test)

In [91]:
X_test.shape

(7500, 78)

y_train and y_test have to be one-hot-encoded. That means they must have dimension (number_of_samples, 15), where 15 denotes number of classes.

In [92]:
y_vector = y_train.values.tolist()
print(len(y_vector))
y_test_v = y_test.values.tolist()
print(len(y_test_v))

30000
7500


In [93]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_vector, 15)
y_test = to_categorical(y_test_v, 15)
print(y_train.shape)
print(y_test.shape)

(30000, 15)
(7500, 15)


## Step 3. Build the model

In [94]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm

In [104]:
#hyper-params
batch_size = 512 # increasing batch size with more gpu added

input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 90
learning_rates = 0.0001

optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
print(input_dim)
print(num_class)

78
15


In [96]:
#X_train_r = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(30000, 78, 1)


In [97]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(7500, 78, 1)


In [105]:
model = Sequential()

# input layer
model.add(Conv1D(filters=64, kernel_size=11, padding='same', input_shape=(78,1)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=7))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=256, kernel_size=5))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=512, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=256, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_class))
model.add(Activation('softmax'))


model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_30 (Conv1D)           (None, 78, 64)            768       
_________________________________________________________________
batch_normalization_30 (Batc (None, 78, 64)            312       
_________________________________________________________________
activation_36 (Activation)   (None, 78, 64)            0         
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 39, 64)            0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 33, 128)           57472     
_________________________________________________________________
batch_normalization_31 (Batc (None, 33, 128)           132       
_________________________________________________________________
activation_37 (Activation)   (None, 33, 128)          

In [106]:
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']) 

## Step 4. Training the model

In [107]:
# fit network
model.fit(X_train_r, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_r, y_test), verbose=1)

Epoch 1/90
59/59 [==============================] - 4s 38ms/step - loss: 1.9602 - accuracy: 0.3986 - val_loss: 2.6803 - val_accuracy: 0.0667
Epoch 2/90
59/59 [==============================] - 2s 27ms/step - loss: 0.6230 - accuracy: 0.7734 - val_loss: 2.6102 - val_accuracy: 0.0832
Epoch 3/90
59/59 [==============================] - 2s 27ms/step - loss: 0.4605 - accuracy: 0.8158 - val_loss: 2.4205 - val_accuracy: 0.1148
Epoch 4/90
59/59 [==============================] - 2s 28ms/step - loss: 0.3863 - accuracy: 0.8385 - val_loss: 2.1189 - val_accuracy: 0.2979
Epoch 5/90
59/59 [==============================] - 2s 27ms/step - loss: 0.3328 - accuracy: 0.8517 - val_loss: 1.7134 - val_accuracy: 0.4192
Epoch 6/90
59/59 [==============================] - 2s 28ms/step - loss: 0.3129 - accuracy: 0.8587 - val_loss: 1.2378 - val_accuracy: 0.6628
Epoch 7/90
59/59 [==============================] - 2s 28ms/step - loss: 0.3119 - accuracy: 0.8554 - val_loss: 0.8796 - val_accuracy: 0.7543
Epoch 8/90
59

In [108]:
# evaluate model
accuracy = model.evaluate(X_test_r, y_test, batch_size=batch_size, verbose=1)

15/15 [==============================] - 0s 10ms/step - loss: 0.7558 - accuracy: 0.7829


## Step 5. Reorder the features and put the most important feature first

In [15]:
column_names = ['colA',
 'Dst Port',
 'Protocol',
 'Flow Duration',
 'Tot Fwd Pkts',
 'Tot Bwd Pkts',
 'TotLen Fwd Pkts',
 'TotLen Bwd Pkts',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Min',
 'Fwd Pkt Len Mean',
 'Fwd Pkt Len Std',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Min',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Std',
 'Flow Byts/s',
 'Flow Pkts/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Flow IAT Min',
 'Fwd IAT Tot',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Fwd IAT Min',
 'Bwd IAT Tot',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'Fwd Header Len',
 'Bwd Header Len',
 'Fwd Pkts/s',
 'Bwd Pkts/s',
 'Pkt Len Min',
 'Pkt Len Max',
 'Pkt Len Mean',
 'Pkt Len Std',
 'Pkt Len Var',
 'FIN Flag Cnt',
 'SYN Flag Cnt',
 'RST Flag Cnt',
 'PSH Flag Cnt',
 'ACK Flag Cnt',
 'URG Flag Cnt',
 'CWE Flag Count',
 'ECE Flag Cnt',
 'Down/Up Ratio',
 'Pkt Size Avg',
 'Fwd Seg Size Avg',
 'Bwd Seg Size Avg',
 'Fwd Byts/b Avg',
 'Fwd Pkts/b Avg',
 'Fwd Blk Rate Avg',
 'Bwd Byts/b Avg',
 'Bwd Pkts/b Avg',
 'Bwd Blk Rate Avg',
 'Subflow Fwd Pkts',
 'Subflow Fwd Byts',
 'Subflow Bwd Pkts',
 'Subflow Bwd Byts',
 'Init Fwd Win Byts',
 'Init Bwd Win Byts',
 'Fwd Act Data Pkts',
 'Fwd Seg Size Min',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Active Min',
 'Idle Mean',
 'Idle Std',
 'Idle Max',
 'Idle Min']

In [109]:
len(column_names)

79

According to "**Selection and Performance Analysis of CICIDS2017 Features Importance**", the important features are: *Destination Port, Fwd IAT Min, Init_Win_bytes_Forward, Init_Win_bytes_backward* and *FlowIATMin*

In [46]:
important_column_names = ['Dst Port',
 'Fwd IAT Min',
 'Init Fwd Win Byts',
 'Init Bwd Win Byts',
 'Flow IAT Min',
 'Flow Duration',
 'Tot Fwd Pkts',
 'Tot Bwd Pkts',
 'TotLen Fwd Pkts',
 'TotLen Bwd Pkts',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Min',
 'Fwd Pkt Len Mean',
 'Fwd Pkt Len Std',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Min',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Std',
 'Flow Byts/s',
 'Flow Pkts/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Fwd IAT Tot',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Bwd IAT Tot',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'Fwd Header Len',
 'Bwd Header Len',
 'Fwd Pkts/s',
 'Bwd Pkts/s',
 'Pkt Len Min',
 'Pkt Len Max',
 'Pkt Len Mean',
 'Pkt Len Std',
 'Pkt Len Var',
 'FIN Flag Cnt',
 'SYN Flag Cnt',
 'RST Flag Cnt',
 'PSH Flag Cnt',
 'ACK Flag Cnt',
 'URG Flag Cnt',
 'CWE Flag Count',
 'ECE Flag Cnt',
 'Down/Up Ratio',
 'Pkt Size Avg',
 'Fwd Seg Size Avg',
 'Bwd Seg Size Avg',
 'Fwd Byts/b Avg',
 'Fwd Pkts/b Avg',
 'Fwd Blk Rate Avg',
 'Bwd Byts/b Avg',
 'Bwd Pkts/b Avg',
 'Bwd Blk Rate Avg',
 'Subflow Fwd Pkts',
 'Subflow Fwd Byts',
 'Subflow Bwd Pkts',
 'Subflow Bwd Byts',
 'Fwd Act Data Pkts',
 'Fwd Seg Size Min',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Active Min',
 'Idle Mean',
 'Idle Std',
 'Idle Max',
 'Idle Min',
  'Protocol',
  'colA']

Read X_train and X_test datasets.

In [110]:
X_train = pd.read_csv(cleanfile, skiprows=0,index_col=0) 
X_test = pd.read_csv(xtestsmall, skiprows=0, index_col=0)
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,1755722.0,80.0,6.0,9743.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,205.275583,9743.000000,0.000000e+00,9743.0,9743.0,9743.0,9743.00,0.000000e+00,9743.0,9743.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,205.275583,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6156600.0,445.0,6.0,111085.0,6.0,5.0,455.0,338.0,140.0,0.0,75.833333,62.936211,145.0,0.0,67.600000,69.758870,7138.677589,99.023270,11108.500000,1.422834e+04,29284.0,4.0,111051.0,22210.20,1.245618e+04,29356.0,4.0,110986.0,27746.500000,1196.308349,29371.0,26744.0,0.0,0.0,0.0,0.0,132.0,112.0,54.012693,45.010577,0.0,145.0,66.083333,63.402693,4019.901515,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,72.090909,75.833333,67.600000,0.0,0.0,0.0,0.0,0.0,0.0,6.0,455.0,5.0,338.0,8192.0,0.0,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,242193.0,8080.0,6.0,10686.0,3.0,4.0,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,0.0,32.250000,53.767245,42579.075430,655.062699,1781.000000,3.912594e+03,9758.0,21.0,563.0,281.50,3.245620e+02,511.0,52.0,10196.0,3398.666667,5510.518064,9758.0,32.0,0.0,0.0,0.0,0.0,72.0,92.0,280.741157,374.321542,0.0,326.0,56.875000,115.406657,13318.696430,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,65.000000,108.666667,32.250000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,326.0,4.0,129.0,8192.0,219.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6934977.0,22.0,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,285714.285714,7.000000,0.000000e+00,7.0,7.0,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,32.0,142857.142857,142857.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,241.0,230.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4505141.0,80.0,6.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,16.0,5007525.0,1251881.25,2.467885e+06,4953524.0,406.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,8192.0,221.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Add the column names to them

In [111]:
 X_train.columns =  column_names
 X_test.columns = column_names
 X_train.head()

,colA,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,1755722.0,80.0,6.0,9743.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,205.275583,9743.000000,0.000000e+00,9743.0,9743.0,9743.0,9743.00,0.000000e+00,9743.0,9743.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,205.275583,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6156600.0,445.0,6.0,111085.0,6.0,5.0,455.0,338.0,140.0,0.0,75.833333,62.936211,145.0,0.0,67.600000,69.758870,7138.677589,99.023270,11108.500000,1.422834e+04,29284.0,4.0,111051.0,22210.20,1.245618e+04,29356.0,4.0,110986.0,27746.500000,1196.308349,29371.0,26744.0,0.0,0.0,0.0,0.0,132.0,112.0,54.012693,45.010577,0.0,145.0,66.083333,63.402693,4019.901515,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,72.090909,75.833333,67.600000,0.0,0.0,0.0,0.0,0.0,0.0,6.0,455.0,5.0,338.0,8192.0,0.0,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,242193.0,8080.0,6.0,10686.0,3.0,4.0,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,0.0,32.250000,53.767245,42579.075430,655.062699,1781.000000,3.912594e+03,9758.0,21.0,563.0,281.50,3.245620e+02,511.0,52.0,10196.0,3398.666667,5510.518064,9758.0,32.0,0.0,0.0,0.0,0.0,72.0,92.0,280.741157,374.321542,0.0,326.0,56.875000,115.406657,13318.696430,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,65.000000,108.666667,32.250000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,326.0,4.0,129.0,8192.0,219.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6934977.0,22.0,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,285714.285714,7.000000,0.000000e+00,7.0,7.0,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,32.0,142857.142857,142857.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,241.0,230.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4505141.0,80.0,6.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,16.0,5007525.0,1251881.25,2.467885e+06,4953524.0,406.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,8192.0,221.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


reorder the columns

In [112]:
X_train_important = X_train.reindex(important_column_names, axis=1)
X_test_important = X_test.reindex(important_column_names, axis=1)
X_train_important.head()

,Dst Port,Fwd IAT Min,Init Fwd Win Byts,Init Bwd Win Byts,Flow IAT Min,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol,colA
0,80.0,9743.0,225.0,-1.0,9743.0,9743.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,205.275583,9743.000000,0.000000e+00,9743.0,9743.0,9743.00,0.000000e+00,9743.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,205.275583,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1755722.0
1,445.0,4.0,8192.0,0.0,4.0,111085.0,6.0,5.0,455.0,338.0,140.0,0.0,75.833333,62.936211,145.0,0.0,67.600000,69.758870,7138.677589,99.023270,11108.500000,1.422834e+04,29284.0,111051.0,22210.20,1.245618e+04,29356.0,110986.0,27746.500000,1196.308349,29371.0,26744.0,0.0,0.0,0.0,0.0,132.0,112.0,54.012693,45.010577,0.0,145.0,66.083333,63.402693,4019.901515,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,72.090909,75.833333,67.600000,0.0,0.0,0.0,0.0,0.0,0.0,6.0,455.0,5.0,338.0,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6156600.0
2,8080.0,52.0,8192.0,219.0,21.0,10686.0,3.0,4.0,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,0.0,32.250000,53.767245,42579.075430,655.062699,1781.000000,3.912594e+03,9758.0,563.0,281.50,3.245620e+02,511.0,10196.0,3398.666667,5510.518064,9758.0,32.0,0.0,0.0,0.0,0.0,72.0,92.0,280.741157,374.321542,0.0,326.0,56.875000,115.406657,13318.696430,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,65.000000,108.666667,32.250000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,326.0,4.0,129.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,242193.0
3,22.0,0.0,241.0,230.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,285714.285714,7.000000,0.000000e+00,7.0,0.0,0.00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,32.0,142857.142857,142857.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6934977.0
4,80.0,406.0,8192.0,221.0,16.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,5007525.0,1251881.25,2.467885e+06,4953524.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4505141.0


In [113]:
X_test_important.head()

,Dst Port,Fwd IAT Min,Init Fwd Win Byts,Init Bwd Win Byts,Flow IAT Min,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol,colA
0,80.0,3308.0,225.0,-1.0,3308.0,3308.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,6.045949e+02,3.308000e+03,0.000000e+00,3308.0,3308.0,3308.0,0.000000e+00,3308.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,6.045949e+02,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1369236.0
1,443.0,26.0,8192.0,30016.0,13.0,5049243.0,6.0,4.0,317.0,145.0,235.0,0.0,52.833333,91.693875,145.0,0.0,36.25,72.5,91.498864,1.980495e+00,5.610270e+05,1.654768e+06,4973536.0,5049243.0,1009848.6,2.243429e+06,5023001.0,62698.0,2.089933e+04,2.558980e+04,49684.0,726.0,0.0,0.0,0.0,0.0,132.0,84.0,1.188297e+00,0.792198,0.0,235.0,42.000000,77.722584,6040.800000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,46.200000,52.833333,36.25,0.0,0.0,0.0,0.0,0.0,0.0,6.0,317.0,4.0,145.0,3.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,9129236.0
2,80.0,2543935.0,211.0,219.0,2.0,2543935.0,2.0,1.0,8.0,0.0,8.0,0.0,4.000000,5.656854,0.0,0.0,0.00,0.0,3.144734,1.179275e+00,1.271968e+06,1.798831e+06,2543933.0,2543935.0,2543935.0,0.000000e+00,2543935.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.0,0.0,0.0,0.0,64.0,32.0,7.861836e-01,0.393092,0.0,8.0,4.000000,4.618802,21.333333,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.333333,4.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,1.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,3496568.0
3,52592.0,1.0,510.0,-1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,2.000000e+06,1.000000e+00,0.000000e+00,1.0,1.0,1.0,0.000000e+00,1.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,2.000000e+06,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,9198198.0
4,80.0,402.0,8192.0,221.0,16.0,5005365.0,4.0,4.0,646.0,364.0,646.0,0.0,161.500000,323.000000,364.0,0.0,91.00,182.0,201.783486,1.598285e+00,7.150521e+05,1.864579e+06,4943232.0,62133.0,20711.0,3.498673e+04,61110.0,5005341.0,1.668447e+06,2.887131e+06,5002219.0,1015.0,0.0,0.0,0.0,0.0,92.0,92.0,7.991425e-01,0.799143,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,126.250000,161.500000,91.00,0.0,0.0,0.0,0.0,0.0,0.0,4.0,646.0,4.0,364.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4505244.0


Drop the last column.

In [116]:
X_train_important.drop('colA', 1,inplace=True)
X_test_important.drop('colA', 1,inplace=True)
X_train_important.head()

,Dst Port,Fwd IAT Min,Init Fwd Win Byts,Init Bwd Win Byts,Flow IAT Min,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Protocol
0,80.0,9743.0,225.0,-1.0,9743.0,9743.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,205.275583,9743.000000,0.000000e+00,9743.0,9743.0,9743.00,0.000000e+00,9743.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,205.275583,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
1,445.0,4.0,8192.0,0.0,4.0,111085.0,6.0,5.0,455.0,338.0,140.0,0.0,75.833333,62.936211,145.0,0.0,67.600000,69.758870,7138.677589,99.023270,11108.500000,1.422834e+04,29284.0,111051.0,22210.20,1.245618e+04,29356.0,110986.0,27746.500000,1196.308349,29371.0,26744.0,0.0,0.0,0.0,0.0,132.0,112.0,54.012693,45.010577,0.0,145.0,66.083333,63.402693,4019.901515,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,72.090909,75.833333,67.600000,0.0,0.0,0.0,0.0,0.0,0.0,6.0,455.0,5.0,338.0,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,8080.0,52.0,8192.0,219.0,21.0,10686.0,3.0,4.0,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,0.0,32.250000,53.767245,42579.075430,655.062699,1781.000000,3.912594e+03,9758.0,563.0,281.50,3.245620e+02,511.0,10196.0,3398.666667,5510.518064,9758.0,32.0,0.0,0.0,0.0,0.0,72.0,92.0,280.741157,374.321542,0.0,326.0,56.875000,115.406657,13318.696430,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,65.000000,108.666667,32.250000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,326.0,4.0,129.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
3,22.0,0.0,241.0,230.0,7.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,285714.285714,7.000000,0.000000e+00,7.0,0.0,0.00,0.000000e+00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,32.0,142857.142857,142857.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,80.0,406.0,8192.0,221.0,16.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,5007525.0,1251881.25,2.467885e+06,4953524.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


## Step 6. Normalization Again

In [117]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_important)
X_test = scaler.fit_transform(X_test_important)

In [118]:
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]

In [119]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]

## Step 7. Reuse the Model and train it again

In [65]:
model = Sequential()

# input layer
model.add(Conv1D(filters=64, kernel_size=11, padding='same', input_shape=(79,1)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=7))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=256, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=512, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=256, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_class))
model.add(Activation('softmax'))


model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_20 (Conv1D)           (None, 79, 64)            768       
_________________________________________________________________
batch_normalization_20 (Batc (None, 79, 64)            316       
_________________________________________________________________
activation_24 (Activation)   (None, 79, 64)            0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 39, 64)            0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 33, 128)           57472     
_________________________________________________________________
batch_normalization_21 (Batc (None, 33, 128)           132       
_________________________________________________________________
activation_25 (Activation)   (None, 33, 128)          

In [120]:
#num_epochs = 35

model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']) 

In [121]:
model.fit(X_train_r, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_r, y_test), verbose=1)

Epoch 1/90
59/59 [==============================] - 3s 33ms/step - loss: 1.6398 - accuracy: 0.5754 - val_loss: 0.4154 - val_accuracy: 0.8356
Epoch 2/90
59/59 [==============================] - 2s 27ms/step - loss: 0.3417 - accuracy: 0.8507 - val_loss: 0.3922 - val_accuracy: 0.8225
Epoch 3/90
59/59 [==============================] - 2s 27ms/step - loss: 0.3036 - accuracy: 0.8635 - val_loss: 0.4199 - val_accuracy: 0.8264
Epoch 4/90
59/59 [==============================] - 2s 28ms/step - loss: 0.2766 - accuracy: 0.8691 - val_loss: 0.3884 - val_accuracy: 0.8424
Epoch 5/90
59/59 [==============================] - 2s 28ms/step - loss: 0.2690 - accuracy: 0.8731 - val_loss: 0.3944 - val_accuracy: 0.8291
Epoch 6/90
59/59 [==============================] - 2s 28ms/step - loss: 0.2645 - accuracy: 0.8718 - val_loss: 0.3874 - val_accuracy: 0.8325
Epoch 7/90
59/59 [==============================] - 2s 28ms/step - loss: 0.2571 - accuracy: 0.8741 - val_loss: 0.3934 - val_accuracy: 0.8369
Epoch 8/90
59

In [78]:
learning_rates = 1e-2

optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']) 

In [79]:
model.fit(X_train_r, y_train, epochs=60, batch_size=batch_size, validation_data=(X_test_r, y_test), verbose=1)

Epoch 1/60
59/59 [==============================] - 3s 35ms/step - loss: 2.5556 - accuracy: 0.3941 - val_loss: 61.0367 - val_accuracy: 0.1635
Epoch 2/60
59/59 [==============================] - 2s 29ms/step - loss: 0.6401 - accuracy: 0.7750 - val_loss: 15.4267 - val_accuracy: 0.2452
Epoch 3/60
59/59 [==============================] - 2s 30ms/step - loss: 0.4482 - accuracy: 0.8269 - val_loss: 5.9329 - val_accuracy: 0.3504
Epoch 4/60
59/59 [==============================] - 2s 30ms/step - loss: 0.2946 - accuracy: 0.8786 - val_loss: 2.8844 - val_accuracy: 0.4455
Epoch 5/60
59/59 [==============================] - 2s 30ms/step - loss: 0.3648 - accuracy: 0.8599 - val_loss: 4.0173 - val_accuracy: 0.4439
Epoch 6/60
59/59 [==============================] - 2s 30ms/step - loss: 0.2362 - accuracy: 0.8961 - val_loss: 1.4192 - val_accuracy: 0.6429
Epoch 7/60
59/59 [==============================] - 2s 30ms/step - loss: 0.2220 - accuracy: 0.9045 - val_loss: 1.1894 - val_accuracy: 0.7513
Epoch 8/60
